In [1]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 38.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819467 sha256=7abbcb696262abff3566bcf05733a76bd2d812daabf44e854d58eb2d3c629886
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4f85256907fb7f9b83e709c65d8a567ca847fc79e2bf77222c2a63f089f72a31
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e74629

In [2]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/tiktok/tiktok_dataset.csv', index_col=['#'])
df.head()

,claim_status,video_id,video_duration_sec,video_transcription_text,verified_status,author_ban_status,video_view_count,video_like_count,video_share_count,video_download_count,video_comment_count
#,,,,,,,,,,,
1,claim,7017666017,59,someone shared with me that drone deliveries a...,not verified,under review,343296.0,19425.0,241.0,1.0,0.0
2,claim,4014381136,32,someone shared with me that there are more mic...,not verified,active,140877.0,77355.0,19034.0,1161.0,684.0
3,claim,9859838091,31,someone shared with me that american industria...,not verified,active,902185.0,97690.0,2858.0,833.0,329.0
4,claim,1866847991,25,someone shared with me that the metro of st. p...,not verified,active,437506.0,239954.0,34812.0,1234.0,584.0
5,claim,7105231098,19,someone shared with me that the number of busi...,not verified,active,56167.0,34987.0,4110.0,547.0,152.0


In [3]:
from bertopic import BERTopic

topic_model = BERTopic(verbose=True, top_n_words=20, nr_topics=100)
topics, probs = topic_model.fit_transform(df['video_transcription_text'].dropna().values)

/opt/conda/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/conda/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argume

Batches:   0%|          | 0/597 [00:00<?, ?it/s]

2023-10-19 17:09:41,071 - BERTopic - Transformed documents to Embeddings
2023-10-19 17:10:26,344 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-10-19 17:10:28,801 - BERTopic - Clustered reduced embeddings
2023-10-19 17:10:30,599 - BERTopic - Reduced number of topics from 593 to 100


In [4]:
from collections import Counter
# print('BERTopic thinks we have a lot of topics. How many topics? {} topics.'.format(len(set(topics))))
print('How many topics? {} topics.'.format(len(set(topics))))
topic_sizes = dict(Counter(topics))
topic_size_values = list(topic_sizes.values())

print('The mean topic size is {:5.2f} documents.'.format(sum(topic_size_values)/len(topic_size_values)))

How many topics? 100 topics.
The mean topic size is 190.84 documents.


In [5]:
from plotly.express import bar
bar(x=list(topic_sizes.keys()), y=list(topic_sizes.values()), log_y=True)

In [6]:
# let's just take a sample and look at the similarities; ideally we would sort the topic by similarity
topic_model.visualize_heatmap(top_n_topics=30)

In [7]:
hierarchical_topics = topic_model.hierarchical_topics(df['video_transcription_text'].dropna().values)

100%|██████████| 98/98 [00:00<00:00, 108.19it/s]


In [8]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [9]:
topic_model.visualize_topics(width=1000, height=1000)

Even though we are arbitrarily reducing our topics from 600 or 601 to 100 we still see some topics that are very near one another in this representation.